In [1]:
import sys;sys.path.append('..')
from ppanlp.corpus import *
from ppanlp.imports import *

In [2]:
ppa = PPA('~/ppa_data/corpus2')

In [3]:
ppa.gen(num_proc=7,force=True)

Saving cleaned pages into page db [x7]:  80%|███████▉  | 5531/6939 [17:53<04:33,  5.15it/s]  


In [4]:
!pip install -U pip wheel
!pip install pymongolite

  Using cached pymongolite-0.1.6-py3-none-any.whl (25 kB)
  Using cached blist-1.3.6.tar.gz (122 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [38 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-13.5-x86_64-cpython-310
      creating build/lib.macosx-13.5-x86_64-cpython-310/blist
      copying blist/_btuple.py -> build/lib.macosx-13.5-x86_64-cpython-310/blist
      copying blist/_sortedlist.py -> build/lib.macosx-13.5-x86_64-cpython-310/blist
      copying blist/__init__.py -> build/lib.macosx-13.5-x86_64-cpython-310/blist
      copying blist/_sorteddict.py -> build/lib.macosx-13.5-x86_64-cpython-310/blist
      running build_ext
      building 'blist._blist' extension
      creating build/temp.macosx-13.5-x86_64-cpython-310
      creating build/temp.macosx-13.5-x86_64-cpython-

In [4]:
objs = ppa.text_ids

def func(obj):
    t = PPA().textd[obj]
    t.gen(force=True)

In [5]:
pmap_run(func, objs)

Iteration over texts in PPA: 100%|██████████| 6939/6939 [00:00<00:00, 157479.59it/s]

Iteration over texts in PPA: 100%|██████████| 6939/6939 [00:00<00:00, 152117.97it/s]








Iteration over texts in PPA: 100%|██████████| 6939/6939 [00:00<00:00, 161586.29it/s]


Mapping func() [x14]:  15%|█▍        | 1019/6939 [00:17<01:22, 71.66it/s]

In [ ]:
# def iter_pages(self, work_ids=None, clean=None, lim=None, min_doc_len=None, frac=1, frac_text=1, max_per_cluster=None, as_dict=True):
#     if not frac or frac>1 or frac<=0: frac=1
#     # if not frac_text or frac_text>1 or frac_text<=0: frac_text=1
#     ppa.textd
#     i=0
#     clustd=Counter()
#     q={}
#     if frac and frac>0 and frac<1: q['_random']={'$lte':frac}
#     if work_ids: q['work_id']={'$in':list(work_ids)}
#     if min_doc_len: q['page_num_content_words']={'$gte':min_doc_len}
#     for d in tqdm(self.page_db.find(q),desc='Iterating over page search results'):
#         del d['_id']
#         t=self.textd[d['work_id']]
#         if not max_per_cluster or clustd[t.cluster]<max_per_cluster:
#             yield PPAPage(d['page_id'], t, **d) if not as_dict else d
#             if max_per_cluster: clustd[t.cluster]+=1
#             i+=1
#             if lim and i>=lim: break
#         if lim and i>=lim: break


In [ ]:
# f=iter_pages(ppa, max_per_cluster=100, min_doc_len=25, as_dict=True,frac=.1)
# df=pd.DataFrame(f)
# df